In [1]:
from ultralytics import YOLO
import cv2
import torch
import numpy as np

In [2]:
def segmentation_to_bbox(masks):
    segmentation = np.where(masks)

    # Bounding Box
    bbox = 0, 0, 0, 0
    if len(segmentation) != 0 and len(segmentation[1]) != 0 and len(segmentation[0]) != 0:
        x_min = int(np.min(segmentation[1]))
        x_max = int(np.max(segmentation[1]))
        y_min = int(np.min(segmentation[0]))
        y_max = int(np.max(segmentation[0]))

        bbox = x_min, x_max, y_min, y_max

    return bbox


def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

In [3]:
model = YOLO('best.pt')

In [22]:
filepath = "datasets/images/val/" + "5879e775-DataTrain1.png"

In [23]:

results = model(filepath, imgsz=640)
img = cv2.imread(filepath)

for result in results:
    for mask in result.masks:
        dim = mask.data.shape
        img = cv2.resize(img, (dim[2], dim[1]))

        m = torch.squeeze(mask.data)
        composite = torch.stack((m, m, m), 2)
        tmp = img * composite.cpu().numpy().astype(np.uint8)
        crop = tmp
        cv2.imshow("original", img)
        cv2.imshow("result", tmp)
        cv2.waitKey(0)
        break
    break
cv2.destroyAllWindows()


image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\datasets\images\val\5879e775-DataTrain1.png: 224x640 2 License Plates, 157.8ms
Speed: 3.5ms preprocess, 157.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


In [24]:
masks = results[0].masks[0].data[0].cpu().numpy()

In [25]:
cropped = crop.copy()
gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
canny = cv2.Canny(gray, 20, 50)
kernel = np.ones((3, 3), np.uint8)
dilation = cv2.dilate(canny, kernel, iterations=2)
erosion = cv2.erode(dilation, kernel, iterations=1)


cv2.imshow("erosion", erosion)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
_, thresh = cv2.threshold(erosion, 45, 255, cv2.THRESH_BINARY)
cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)

In [27]:
# find biggest contour
c = max(cnts, key=cv2.contourArea)

peri = cv2.arcLength(c, True)
approx = cv2.approxPolyDP(c, 0.02 * peri, True)

In [28]:
circle = cropped.copy()
for i in range(len(approx)):
    cv2.circle(circle, (approx[i][0][0], approx[i][0][1]), 3, (0, 255, 0), -1)

cv2.imshow("circle", circle)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
perspective = []
rect = approx
for i in range(4):
    point_before = approx[i % 4][0]
    perspective.append((point_before[0], point_before[1]))
    point_after = approx[(i + 1) % 4][0]
    cv2.line(cropped, point_before, point_after, (0, 255, 0), 2)

perspective = order_points(np.array(perspective))

cv2.imshow("original", img)
cv2.imshow("cropped", cropped)

height, width, channels = cropped.shape

# pts1 = np.float32([[0, 0], [width - 150, 0], [width, height], [0, height]])
pts1 = np.float32(perspective)
pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)
transformed = cv2.warpPerspective(cropped, matrix, (width, height))

cv2.imshow("transformed", transformed)
cv2.waitKey(0)
cv2.destroyAllWindows()